In [1]:
import os
import inspect
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import compiling, rasp_to_graph
import numpy as np
from tracr.compiler.validating import validate
from rasp_generator import map_primitives, sampling, utils
import networkx as nx

from tracr.rasp.rasp import Map, SequenceMap, LinearSequenceMap, Select, Aggregate, Comparison, SelectorWidth, indices, tokens 


def compile_rasp_to_model(sop: rasp.SOp, vocab={1,2,3,4}, max_seq_len=5, compiler_bos="BOS"):
    return compiling.compile_rasp_to_model(
        sop,
        vocab=vocab,
        max_seq_len=max_seq_len,
        compiler_bos=compiler_bos
    )


TEST_INPUTS = [
    [1,2,3,4],
    [0,1,2,3,4],
    [4,3,2,1,0],
    [0,3,2,4],
    [1,1,1,1],
    [0,0,0,0],
    [4,4,4,4],
    [0,0,4,0]
]

# Program Generation

In [8]:
sampler = sampling.ProgramSampler(validate_compilation=True)
errs = sampler.sample(n_sops=12)
print("Retries encountered:")
print()
for err in errs:
    print(err)
print()
print("Program outputs:")
for x in TEST_INPUTS:
    print(sampler.program(x))

SamplingError: Failed to sample program. Received errors:
Filter failed. No SOps in scope satisfy constraints.
Filter failed. No SOps in scope satisfy constraints.
Filter failed. No SOps in scope satisfy constraints.
Filter failed. No SOps in scope satisfy constraints.

In [3]:
# print program with and without test input
k = 1
utils.print_program(sampler.sops[-k])
print()
print()
utils.print_program(sampler.sops[-k], TEST_INPUTS[0])

sequence_map_1 = SequenceMap(lambda x, y: x*y, tokens, indices)    # type: categorical


sequence_map_1 = SequenceMap(lambda x, y: x*y, tokens, indices)    # output: [0, 2, 6, 12]
